In [18]:
from Pull_ROI_Functional import *
import pandas as pd
import os

In [19]:

func_img = nib.load('/Volumes/AUERBACHLAB/Columbia/HCP_BANDA_1.0_Data/NDA_FINAL/derivatives/BANDA003_MR/func/faces/BANDA003_MR_task-face_level2.gfeat/cope7.feat/stats/tstat1.nii.gz')


In [20]:
ROI_dataframe = pd.read_csv('tapasi_roi_list_without_Reuniens.csv')

In [21]:
for i, row in ROI_dataframe.iterrows():
    if row['HEMISPHERE'] == 'LEFT HEMISPHERE':
        ROI_dataframe.loc[i, 'RegionLongName'] = row['RegionLongName'] + '_Left'
    else:
        ROI_dataframe.loc[i, 'RegionLongName'] = row['RegionLongName'] + '_Right'

In [22]:
#185 participants
Clinical_Data_Clean_for_T2 = pd.read_excel('/Users/eastvillage/Desktop/Tapasi/Model/Clinical_Data_Clean_for_T2.xlsx')


In [23]:
#182 participants
Clinical_Structural_Data_Clean = pd.read_excel('/Users/eastvillage/Desktop/Tapasi/Model/Clinical_Structural_Data_Clean.xlsx')


In [24]:
ROI_array_for_columns = np.insert(np.array(ROI_dataframe['RegionLongName']),0,'src_subject_id')
Functional_data = pd.DataFrame(columns=ROI_array_for_columns)

participant_path = '/Volumes/AUERBACHLAB/Columbia/HCP_BANDA_1.0_Data/NDA_FINAL/derivatives/'

# Loop through num. of participants times and add a new row in each iteration
for j in range(len(Clinical_Data_Clean_for_T2)):
    # Create a new row as a dictionary
    #6 participants from the clean T2 data do not have fMRI data (185 - 179)
    if Clinical_Data_Clean_for_T2['src_subject_id'][j]+'_MR' in os.listdir(participant_path):
        new_row = {ROI_array_for_columns[0]: Clinical_Data_Clean_for_T2['src_subject_id'][j]} #Entering participant ID in each row
        
        column_num = 1
        for i in ROI_dataframe['ID']:

            output = pull_roi(func_path = participant_path + Clinical_Data_Clean_for_T2['src_subject_id'][j] +'_MR/func/faces/'+ Clinical_Data_Clean_for_T2['src_subject_id'][j] +'_MR_task-face_level2.gfeat/cope7.feat/stats/tstat1.nii.gz',
                    label_path = '/Volumes/AUERBACHLAB/Columbia/General/Tapasi_Brahma/Atlas/HCPex/HCPex_FSLspace.nii.gz',
                    thresh_value = i)

            new_row.update({ROI_array_for_columns[column_num]: output}) 
            column_num = column_num + 1
            
        Functional_data = pd.concat([Functional_data, (pd.Series(new_row).to_frame().T)], ignore_index=True)

In [ ]:
# #List of missing fMRI data
# c = 0

# for j in range(len(Clinical_Data_Clean_for_T2)):
#     # Create a new row as a dictionary
#     if not(str(Clinical_Data_Clean_for_T2['src_subject_id'][j]+'_MR') in os.listdir(participant_path)):
        
#         print(str(Clinical_Data_Clean_for_T2['src_subject_id'][j]+'_MR'))
        
#         c = c+1
# print(c)
        

In [25]:
Functional_data.to_excel('/Users/eastvillage/Desktop/Tapasi/Model/Prediction of T2 scores/Functional_MRI/Functional_data.xlsx', index=False)


In [28]:
#Testing for NaN entries:Mean of empty slice.:invalid value encountered in scalar divide
NaN_participant_num = 0
for participant in np.array(Functional_data['src_subject_id']):
    
    nan_columns = Functional_data.loc[Functional_data['src_subject_id'] == participant].isna().any()

    # Print the column names where NaN values were found
    if nan_columns.any():
        print("For ", participant," NaN values were found in the following columns:")
        for col_name in nan_columns[nan_columns].index:
            print(col_name)
            print("\n")
            
        NaN_participant_num = NaN_participant_num + 1
    else:
        print("There are no NaN values in the selected columns.")
        
print("Number of participants with NaN entries = ",NaN_participant_num)

There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN values in the selected columns.
There are no NaN val

In [29]:
# #Checking
# for i in ROI_dataframe['ID']:
#     output = pull_roi(func_path = '/Volumes/AUERBACHLAB/Columbia/HCP_BANDA_1.0_Data/NDA_FINAL/derivatives/BANDA211_MR/func/faces/BANDA211_MR_task-face_level2.gfeat/cope7.feat/stats/tstat1.nii.gz',
#             label_path = '/Volumes/AUERBACHLAB/Columbia/General/Tapasi_Brahma/Atlas/HCPex/HCPex_FSLspace.nii.gz',
#             thresh_value = i)
    
#     print(i, output)
    

In [39]:
#Concatanating (Clinical, Demographic), and functional dataframes:
#179 participants

Demographic_Clinical_Functional_T2 = pd.merge(Clinical_Data_Clean_for_T2, Functional_data, on='src_subject_id', how='inner')


In [34]:
Demographic_Clinical_Functional_T2.to_excel('/Users/eastvillage/Desktop/Tapasi/Model/Prediction of T2 scores/Functional_MRI/Demographic_Clinical_Functional_T2.xlsx', index=False)


In [35]:
#Concatanating (Clinical, Demographic), structural, and functional dataframes:
#179 participants

Demographic_Clinical_Structural_Functional_T2 = pd.merge(Clinical_Structural_Data_Clean, Functional_data, on='src_subject_id', how='inner')


In [36]:
Demographic_Clinical_Structural_Functional_T2.shape #11 Structural ROIs - 'Unnamed: 0'

(178, 157)

In [40]:
Demographic_Clinical_Functional_T2.shape 

(179, 147)

In [38]:
Demographic_Clinical_Structural_Functional_T2.to_excel('/Users/eastvillage/Desktop/Tapasi/Model/Prediction of T2 scores/Functional_MRI/Demographic_Clinical_Structural_Functional_T2.xlsx', index=False)
